# 태양광 발전량 예측 대회_데이콘

이 대회는 조금 특이합니다. 자세한 건 아래를 참고하세요.<br>
https://medium.com/analytics-vidhya/a-tutorial-on-quantile-regression-quantile-random-forests-and-quantile-gbm-d3c651af7516

핵심은 이 대회는 OLS가 아닌 QR이라고 해서 Quantile Regression을 사용하는 대회입니다. quantile 은 0.1 ~ 0.9 까지 0.1 단위로 9개의 구간입니다.

Loss Function은 Pinball Loss라고 해서 왜 핀볼인지는 모르겠지만 대회를 참고하세요. 아무튼 중요한 것은 분위수가 높으면서 값이 벌어질수록 페널티가 커지고, 분위수가 낮을수록 값이 벌어지면 또 페널티가 커지는. 그냥 정답 값에서 분위수와 동시에 멀어지면 멀어질수록 페널티가 늘어가는 것입니다. 즉 9개의 모델들이 최대한 가운데 뭉쳐서 예측하는 것이 중요합니다.

그리고 또 하나 특이한 점은, train셋은 3년치 데이터가 주어집니다. 그러나 test셋은 7일 단위로 81개가 주어집니다. 총 2년치 데이텁니다. 그러나 시간순이 아니라 랜덤입니다. 즉, 1개를 사용해서 예측해도 좋고, 81개를 다 써서 예측해도 좋습니다. 다만 중요한 건 다음 2일 간의 예측을 하는 것입니다.

pre-trained 모델 금지, 외부데이터 금지

public score는 제출 답의 50% 로 산정, private score는 100%로 산정

7주일치의 기상 데이터를 집어 넣었을 때, 다음 이틀 간의 태양광 발전량을 내뱉어야 합니다. 즉 이것은 어떻게 보면 기상 예측 대회일 수도 있습니다. 또 어찌 보면 피쳐를 시프트해서 예측해야 하는 문제일 수도 있습니다.

- Hour - 시간
- Minute - 분
- DHI - 수평면 산란일사량(Diffuse Horizontal Irradiance (W/m2))
- DNI - 직달일사량(Direct Normal Irradiance (W/m2))
- WS - 풍속(Wind Speed (m/s))
- RH - 상대습도(Relative Humidity (%))
- T - 기온(Temperature (Degree C))
- Target - 태양광 발전량 (kW)

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sktime

In [18]:
print(pd.__version__)
print(np.__version__)
print(sns.__version__)
print(sklearn.__version__)
print(sktime.__version__)

1.0.5
1.19.4
0.10.1
0.23.1
0.4.3


sktime이라는 TS전용 라이브러리입니다. sklearn의 사촌동생입니다.<br>
https://github.com/alan-turing-institute/sktime

In [14]:
TRAIN_PATH = r'C:\Users\Wyatt\wyatt37\Data\solarpanel\train\train.csv'
TEST_PATH = r'C:\Users\Wyatt\wyatt37\Data\solarpanel\test'
SUBMISSION_PATH = r'C:\Users\Wyatt\wyatt37\Data\solarpanel\sample_submission.csv'

In [5]:
pd.read_csv(TRAIN_PATH)

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0


In [20]:
pd.read_csv(TEST_PATH + r'\80.csv')

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,2.1,82.64,3.9,0.0
1,0,0,30,0,0,2.1,83.22,3.8,0.0
2,0,1,0,0,0,2.0,82.28,3.8,0.0
3,0,1,30,0,0,1.8,82.86,3.7,0.0
4,0,2,0,0,0,1.7,82.29,3.6,0.0
...,...,...,...,...,...,...,...,...,...
331,6,21,30,0,0,0.8,63.35,13.7,0.0
332,6,22,0,0,0,0.7,64.82,13.1,0.0
333,6,22,30,0,0,0.7,66.10,12.8,0.0
334,6,23,0,0,0,0.6,67.64,12.4,0.0


In [15]:
pd.read_csv(SUBMISSION_PATH)

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
